In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

In [ ]:
!cd modelscope-agent && git checkout feat/code_interpreter
! pip install -r requirements.txt
! pip install transformers_stream_generator
! pip install invisible-watermark>=2.0

In [ ]:
import os
os.chdir('modelscope-agent/demo')

import sys
sys.path.append('../')

In [ ]:
from dotenv import load_dotenv  
import os
from modelscope.utils.config import Config

# 配置环境变量，里面需要配置modelscope token来remote调用AI模型
os.environ['TOOL_CONFIG_FILE'] ='../config/cfg_tool_template.json'
os.environ['OUTPUT_FILE_DIRECTORY'] = '/tmp'

print('请输入DASHSCOPE_API_KEY')
os.environ['DASHSCOPE_API_KEY'] = input()

# 读取工具调用CONFIG文件，包括工具的名称和远程请求url
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE')
print(tool_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)

In [ ]:
from modelscope_agent.llm import LLMFactory

model_cfg = {
    'qwen_plus':{
        'type': 'dashscope',
        "model": "qwen-plus",
        "generate_cfg": {
            "use_raw_prompt": True,
            "top_p": 0.8,
            "top_k": 10,
            "seed": 123,
            "debug": True
        }
    }
}

model_name = 'qwen_plus'
llm = LLMFactory.build_llm(model_name, model_cfg)

In [ ]:
from modelscope_agent.agent import AgentExecutor
from modelscope_agent.tools import CodeInterpreter
from  modelscope_agent.agent_types import AgentType

# 额外工具列表，这里只有一个code_interpreter
additional_tool_list = {
    'code_interpreter': CodeInterpreter()
}

# 构建Agent，需要传入llm，工具配置config以及工具检索
agent = AgentExecutor(llm, agent_type=AgentType.MRKL, additional_tool_list=additional_tool_list, tool_retrieval=False)
available_tool_list = ['code_interpreter']
agent.set_available_tools(available_tool_list)

In [ ]:
# 重置对话，清空对话历史
agent.reset()

# remote=True为调用modelscope api，该服务免费支持QPS较低，建议部署在本地，将remote=False
agent.run("查看我本地python版本并返回", remote=False, print_info=True)